In [ ]:
import datetime
import pandas as pd
import hopsworks
import os
from dotenv import load_dotenv
from functions import utils

In [2]:
load_dotenv()
HOPSWORKS_API_KEY = os.getenv("HOPSWORKS_API_KEY")

os.environ["HOPSWORKS_API_KEY"] = HOPSWORKS_API_KEY
print(HOPSWORKS_API_KEY)

a7ImTC8DbtMq3SeD.IWdWgtywx4aLmLyy5wYgxGgUhJFvwzeT9EkxyZAeK7GCzR9sVzXflXFjZksLB248


In [3]:
CITY = "dublin"

## Read the bike data from csv-files

In [4]:
# get directory path and all files in it
df_bike = pd.DataFrame()
for file in os.listdir(CITY):
    if file.endswith(".csv"):
        df_tmp = pd.read_csv(f"{CITY}/" + file,  parse_dates=['datetime'], skipinitialspace=True)
        df_tmp['station'] = file.split(".")[0]
        df_bike = pd.concat([df_bike, df_tmp])

df_bike = df_bike.set_index(['datetime', 'station']).reset_index(drop=False)

df_bike

,datetime,station,num_bikes_available
0,2025-01-02 11:05:44+00:00,AVONDALE_ROAD,22
1,2025-01-02 10:48:53+00:00,AVONDALE_ROAD,22
2,2025-01-02 09:49:18+00:00,AVONDALE_ROAD,22
3,2025-01-02 08:49:10+00:00,AVONDALE_ROAD,23
4,2025-01-02 07:49:04+00:00,AVONDALE_ROAD,23
...,...,...,...
1045033,2023-08-05 16:54:54+00:00,YORK_STREET_WEST,20
1045034,2023-08-05 17:56:37+02:00,YORK_STREET_WEST,16
1045035,2023-08-05 14:51:22+00:00,YORK_STREET_WEST,19
1045036,2023-08-05 15:54:31+02:00,YORK_STREET_WEST,17


In [ ]:
df_bike.dropna(inplace=True)
df_bike


,datetime,station,num_bikes_available
0,2025-01-02 11:05:44+00:00,AVONDALE_ROAD,22
1,2025-01-02 10:48:53+00:00,AVONDALE_ROAD,22
2,2025-01-02 09:49:18+00:00,AVONDALE_ROAD,22
3,2025-01-02 08:49:10+00:00,AVONDALE_ROAD,23
4,2025-01-02 07:49:04+00:00,AVONDALE_ROAD,23
...,...,...,...
1045033,2023-08-05 16:54:54+00:00,YORK_STREET_WEST,20
1045034,2023-08-05 17:56:37+02:00,YORK_STREET_WEST,16
1045035,2023-08-05 14:51:22+00:00,YORK_STREET_WEST,19
1045036,2023-08-05 15:54:31+02:00,YORK_STREET_WEST,17


## Get the weather data from Open Mateo

First, we need to get the earliest date of the bike data so that we can get the weater data from Open Mateo for the same period.

In [6]:
earliest_bike_date = pd.Series.min(df_bike['datetime'])
earliest_bike_date = earliest_bike_date.strftime('%Y-%m-%d')
last_bike_date = pd.Series.max(df_bike['datetime'])
last_bike_date = last_bike_date.strftime('%Y-%m-%d')

earliest_bike_date, last_bike_date

('2023-08-05', '2025-01-02')

Then, we get the weather data from Open Mateo

In [ ]:
weather_df = utils.get_historical_weather(CITY, earliest_bike_date, last_bike_date)
weather_df

Coordinates 53.32161331176758°N -6.501922607421875°E
Elevation 11.0 m asl
Timezone None None
Timezone difference to GMT+0 0 s


,date,temperature_2m_mean,apparent_temperature_mean,daylight_duration,rain_sum,snowfall_sum,wind_speed_10m_max,wind_direction_10m_dominant,city
0,2023-08-05,13.722833,11.218610,55572.929688,26.800001,0.0,31.981293,17.058893,dublin
1,2023-08-06,14.066582,12.552302,55362.953125,1.000000,0.0,15.192682,274.469452,dublin
2,2023-08-07,13.935334,12.401314,55149.910156,0.100000,0.0,15.629971,237.169296,dublin
3,2023-08-08,15.570750,15.309894,54933.734375,0.100000,0.0,8.534353,331.534943,dublin
4,2023-08-09,19.114500,19.878794,54714.667969,0.000000,0.0,13.392774,220.062347,dublin
...,...,...,...,...,...,...,...,...,...
511,2024-12-28,8.662417,6.074311,27194.826172,0.000000,0.0,16.671148,224.817642,dublin
512,2024-12-29,8.333251,4.466697,27249.105469,0.000000,0.0,25.331444,229.969803,dublin
513,2024-12-30,9.352000,4.602181,27309.888672,0.000000,0.0,29.533169,220.122574,dublin
514,2024-12-31,10.129084,4.986017,27377.089844,13.000000,0.0,42.379200,222.197067,dublin


## Data validation rules

In [8]:
import great_expectations as ge
aq_expectation_suite = ge.core.ExpectationSuite(
    expectation_suite_name="aq_expectation_suite"
)

aq_expectation_suite.add_expectation(
    ge.core.ExpectationConfiguration(
        expectation_type="expect_column_min_to_be_between",
        kwargs={
            "column":"pm25",
            "min_value":-0.1,
            "max_value":500.0,
            "strict_min":True
        }
    )
)

ModuleNotFoundError: No module named 'great_expectations'